## Regression Tables via `pf.etable()` and the `Stargazer` package

To produce regression tables, we have two options: pyfixest's internal `etable()` function and the [Stargazer](https://github.com/StatsReporting/stargazer) Python package. The following examples illustrate how to use both options.

Contents: 

-   [Regression Tables via `pf.etable()`](#regression-tables-via-pfetable)

    -   [Basic Usage](#basic-usage)
    -   [Keep and drop variables](#keep-and-drop-variables)
    -   [Hide fixed effects or SE-type rows](#hide-fixed-effects-or-se-type-rows)
    -   [Display p-values or confidence intervals](#display-p-values-or-confidence-intervals)
    -   [Significance levels and rounding](#significance-levels-and-rounding)
    -   [Other output formats](#other-output-formats)
    -   [Custom Styling with Great Tables](#custom-styling-with-great-tables)
    -   [Rename variables](#rename-variables)
    -   [Custom model headlines](#custom-model-headlines)
    -   [Further custom model information](#further-custom-model-information)
    -   [Custom table notes](#custom-table-notes)
    -   [Publication-ready LaTex tables](#publication-ready-latex-tables)
    -   [Rendering Tables in Quarto](#rendering-tables-in-quarto)
    
-   [Regression Tables via `Stargazer`](#regression-tables-via-stargazer)


To begin, we load some libraries and fit a set of regression models. 

In [1]:
import numpy as np
import pylatex as pl  # for the latex table; note: not a dependency of pyfixest - needs manual installation
from great_tables import loc, style
from IPython.display import FileLink, display
from stargazer.stargazer import LineLocation, Stargazer

import pyfixest as pf

%load_ext autoreload
%autoreload 2

data = pf.get_data()

fit1 = pf.feols("Y ~ X1 + X2 | f1", data=data)
fit2 = pf.feols("Y ~ X1 + X2 | f1 + f2", data=data)
fit3 = pf.feols("Y ~ X1 *X2 | f1 + f2", data=data)
fit4 = pf.feols("Y2 ~ X1 + X2 | f1", data=data)
fit5 = pf.feols("Y2 ~ X1 + X2 | f1 + f2", data=data)
fit6 = pf.feols("Y2 ~ X1 *X2 | f1 + f2", data=data)

## Regression Tables via `pf.etable()`

### Basic Usage

We can compare all regression models via the pyfixest-internal `pf.etable()` function: 

In [2]:
pf.etable([fit1, fit2, fit3, fit4, fit5, fit6])

GT(_tbl_data=                                    (1)                     (2)  \
0            X1  -0.950*** <br> (0.067)  -0.924*** <br> (0.061)   
1            X2  -0.174*** <br> (0.018)  -0.174*** <br> (0.015)   
2         X1:X2                                                   
3            f1                       x                       x   
4            f2                       -                       x   
5  Observations                     997                     997   
6     S.E. type                  by: f1                  by: f1   
7            R2                   0.489                   0.659   

                      (3)                     (4)                     (5)  \
0  -0.924*** <br> (0.061)  -1.267*** <br> (0.174)  -1.232*** <br> (0.192)   
1  -0.185*** <br> (0.025)   -0.131** <br> (0.042)   -0.118** <br> (0.042)   
2      0.011 <br> (0.018)                                                   
3                       x                       x                       x   
4                       x                       -                       x   
5                     997                     998                     998   
6                  by: f1                  by: f1                  by: f1   
7                   0.659                   0.120                   0.172   

                      (6)     rowtype  
0  -1.231*** <br> (0.192)        coef  
1     -0.074 <br> (0.104)        coef  
2     -0.041 <br> (0.081)        coef  
3                       x          fe  
4                       x          fe  
5                     998  modelstats  
6                  by: f1  modelstats  
7                   0.172  modelstats  , _body=<great_tables._gt_data.Body object at 0x000001A1401B6680>, _boxhead=Boxhead([ColInfo(var='', type=<ColInfoTypeEnum.stub: 2>, column_label='', column_align='left', column_width=None), ColInfo(var='(1)', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='(2)', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='(3)', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='(4)', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='(5)', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='(6)', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None), ColInfo(var='rowtype', type=<ColInfoTypeEnum.row_group: 3>, column_label='rowtype', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001A1401B4D30>, _spanners=Spanners([SpannerInfo(spanner_id='Y', spanner_level=1, spanner_label='Y', spanner_units=None, spanner_pattern=None, vars=['(1)', '(2)', '(3)'], built=None), SpannerInfo(spanner_id='Y2', spanner_level=1, spanner_label='Y2', spanner_units=None, spanner_pattern=None, vars=['(4)', '(5)', '(6)'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001A1401B6380>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='v

### Keep and drop variables
`etable` allows us to do a few things out of the box. For example, we can only keep the variables that we'd like, which keeps all variables that fit the provided regex match. 

In [3]:
pf.etable([fit1, fit2, fit3, fit4, fit5, fit6], keep="X1")

GT(_tbl_data=                                    (1)                     (2)  \
0            X1  -0.950*** <br> (0.067)  -0.924*** <br> (0.061)   
1         X1:X2                                                   
2            f1                       x                       x   
3            f2                       -                       x   
4  Observations                     997                     997   
5     S.E. type                  by: f1                  by: f1   
6            R2                   0.489                   0.659   

                      (3)                     (4)                     (5)  \
0  -0.924*** <br> (0.061)  -1.267*** <br> (0.174)  -1.232*** <br> (0.192)   
1      0.011 <br> (0.018)                                                   
2                       x                       x                       x   
3                       x                       -                       x   
4                     997                     998                     998   
5                  by: f1                  by: f1                  by: f1   
6                   0.659                   0.120                   0.172   

                      (6)     rowtype  
0  -1.231*** <br> (0.192)        coef  
1     -0.041 <br> (0.081)        coef  
2                       x          fe  
3                       x          fe  
4                     998  modelstats  
5                  by: f1  modelstats  
6                   0.172  modelstats  , _body=<great_tables._gt_data.Body object at 0x000001A140197E80>, _boxhead=Boxhead([ColInfo(var='', type=<ColInfoTypeEnum.stub: 2>, column_label='', column_align='left', column_width=None), ColInfo(var='(1)', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='(2)', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='(3)', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='(4)', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='(5)', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='(6)', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None), ColInfo(var='rowtype', type=<ColInfoTypeEnum.row_group: 3>, column_label='rowtype', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001A140196C80>, _spanners=Spanners([SpannerInfo(spanner_id='Y', spanner_level=1, spanner_label='Y', spanner_units=None, spanner_pattern=None, vars=['(1)', '(2)', '(3)'], built=None), SpannerInfo(spanner_id='Y2', spanner_level=1, spanner_label='Y2', spanner_units=None, spanner_pattern=None, vars=['(4)', '(5)', '(6)'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001A140196E30>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu

We can use the `exact_match` argument to select a specific set of variables: 

In [4]:
pf.etable([fit1, fit2, fit3, fit4, fit5, fit6], keep=["X1", "X2"], exact_match=True)

GT(_tbl_data=                                    (1)                     (2)  \
0            X1  -0.950*** <br> (0.067)  -0.924*** <br> (0.061)   
1            X2  -0.174*** <br> (0.018)  -0.174*** <br> (0.015)   
2            f1                       x                       x   
3            f2                       -                       x   
4  Observations                     997                     997   
5     S.E. type                  by: f1                  by: f1   
6            R2                   0.489                   0.659   

                      (3)                     (4)                     (5)  \
0  -0.924*** <br> (0.061)  -1.267*** <br> (0.174)  -1.232*** <br> (0.192)   
1  -0.185*** <br> (0.025)   -0.131** <br> (0.042)   -0.118** <br> (0.042)   
2                       x                       x                       x   
3                       x                       -                       x   
4                     997                     998                     998   
5                  by: f1                  by: f1                  by: f1   
6                   0.659                   0.120                   0.172   

                      (6)     rowtype  
0  -1.231*** <br> (0.192)        coef  
1     -0.074 <br> (0.104)        coef  
2                       x          fe  
3                       x          fe  
4                     998  modelstats  
5                  by: f1  modelstats  
6                   0.172  modelstats  , _body=<great_tables._gt_data.Body object at 0x000001A140194B80>, _boxhead=Boxhead([ColInfo(var='', type=<ColInfoTypeEnum.stub: 2>, column_label='', column_align='left', column_width=None), ColInfo(var='(1)', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='(2)', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='(3)', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='(4)', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='(5)', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='(6)', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None), ColInfo(var='rowtype', type=<ColInfoTypeEnum.row_group: 3>, column_label='rowtype', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001A1401FA710>, _spanners=Spanners([SpannerInfo(spanner_id='Y', spanner_level=1, spanner_label='Y', spanner_units=None, spanner_pattern=None, vars=['(1)', '(2)', '(3)'], built=None), SpannerInfo(spanner_id='Y2', spanner_level=1, spanner_label='Y2', spanner_units=None, spanner_pattern=None, vars=['(4)', '(5)', '(6)'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001A140194130>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu

We can also easily **drop** variables via the `drop` argument: 

In [5]:
pf.etable([fit1, fit2, fit3, fit4, fit5, fit6], drop=["X1"])

GT(_tbl_data=                                    (1)                     (2)  \
0            X2  -0.174*** <br> (0.018)  -0.174*** <br> (0.015)   
1            f1                       x                       x   
2            f2                       -                       x   
3  Observations                     997                     997   
4     S.E. type                  by: f1                  by: f1   
5            R2                   0.489                   0.659   

                      (3)                    (4)                    (5)  \
0  -0.185*** <br> (0.025)  -0.131** <br> (0.042)  -0.118** <br> (0.042)   
1                       x                      x                      x   
2                       x                      -                      x   
3                     997                    998                    998   
4                  by: f1                 by: f1                 by: f1   
5                   0.659                  0.120                  0.172   

                   (6)     rowtype  
0  -0.074 <br> (0.104)        coef  
1                    x          fe  
2                    x          fe  
3                  998  modelstats  
4               by: f1  modelstats  
5                0.172  modelstats  , _body=<great_tables._gt_data.Body object at 0x000001A1401FAF50>, _boxhead=Boxhead([ColInfo(var='', type=<ColInfoTypeEnum.stub: 2>, column_label='', column_align='left', column_width=None), ColInfo(var='(1)', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='(2)', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='(3)', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='(4)', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='(5)', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='(6)', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None), ColInfo(var='rowtype', type=<ColInfoTypeEnum.row_group: 3>, column_label='rowtype', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001A13EB1A710>, _spanners=Spanners([SpannerInfo(spanner_id='Y', spanner_level=1, spanner_label='Y', spanner_units=None, spanner_pattern=None, vars=['(1)', '(2)', '(3)'], built=None), SpannerInfo(spanner_id='Y2', spanner_level=1, spanner_label='Y2', spanner_units=None, spanner_pattern=None, vars=['(4)', '(5)', '(6)'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001A13EB18460>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category=

### Hide fixed effects or SE-type rows
We can hide the rows showing the relevant fixed effects and those showing the S.E. type by setting `show_fe=False` and `show_setype=False` (for instance when the set of fixed effects or the estimation method for the std. errors is the same for all models and you want to describe this in the text or table notes rather than displaying it in the table). 

In [6]:
pf.etable([fit1, fit2, fit3, fit4, fit5, fit6], show_fe=False, show_se_type=False)

GT(_tbl_data=                                    (1)                     (2)  \
0            X1  -0.950*** <br> (0.067)  -0.924*** <br> (0.061)   
1            X2  -0.174*** <br> (0.018)  -0.174*** <br> (0.015)   
2         X1:X2                                                   
3  Observations                     997                     997   
4            R2                   0.489                   0.659   

                      (3)                     (4)                     (5)  \
0  -0.924*** <br> (0.061)  -1.267*** <br> (0.174)  -1.232*** <br> (0.192)   
1  -0.185*** <br> (0.025)   -0.131** <br> (0.042)   -0.118** <br> (0.042)   
2      0.011 <br> (0.018)                                                   
3                     997                     998                     998   
4                   0.659                   0.120                   0.172   

                      (6)     rowtype  
0  -1.231*** <br> (0.192)        coef  
1     -0.074 <br> (0.104)        coef  
2     -0.041 <br> (0.081)        coef  
3                     998  modelstats  
4                   0.172  modelstats  , _body=<great_tables._gt_data.Body object at 0x000001A13EB1A8F0>, _boxhead=Boxhead([ColInfo(var='', type=<ColInfoTypeEnum.stub: 2>, column_label='', column_align='left', column_width=None), ColInfo(var='(1)', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='(2)', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='(3)', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='(4)', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='(5)', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='(6)', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None), ColInfo(var='rowtype', type=<ColInfoTypeEnum.row_group: 3>, column_label='rowtype', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001A1401E8AF0>, _spanners=Spanners([SpannerInfo(spanner_id='Y', spanner_level=1, spanner_label='Y', spanner_units=None, spanner_pattern=None, vars=['(1)', '(2)', '(3)'], built=None), SpannerInfo(spanner_id='Y2', spanner_level=1, spanner_label='Y2', spanner_units=None, spanner_pattern=None, vars=['(4)', '(5)', '(6)'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001A1401E8430>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=Opt

### Display p-values or confidence intervals
By default, `pf.etable()` reports **standard errors**. But we can also ask to output p-values or confidence intervals via the `coef_fmt` function argument. 

In [7]:
pf.etable([fit1, fit2, fit3, fit4, fit5, fit6], coef_fmt="b \n (se) \n [p]")

GT(_tbl_data=                                                 (1)  \
0            X1  -0.950*** <br> (0.067) <br> [0.000]   
1            X2  -0.174*** <br> (0.018) <br> [0.000]   
2         X1:X2                                        
3            f1                                    x   
4            f2                                    -   
5  Observations                                  997   
6     S.E. type                               by: f1   
7            R2                                0.489   

                                   (2)                                  (3)  \
0  -0.924*** <br> (0.061) <br> [0.000]  -0.924*** <br> (0.061) <br> [0.000]   
1  -0.174*** <br> (0.015) <br> [0.000]  -0.185*** <br> (0.025) <br> [0.000]   
2                                           0.011 <br> (0.018) <br> [0.565]   
3                                    x                                    x   
4                                    x                                    x   
5                                  997                                  997   
6                               by: f1                               by: f1   
7                                0.659                                0.659   

                                   (4)                                  (5)  \
0  -1.267*** <br> (0.174) <br> [0.000]  -1.232*** <br> (0.192) <br> [0.000]   
1   -0.131** <br> (0.042) <br> [0.005]   -0.118** <br> (0.042) <br> [0.008]   
2                                                                             
3                                    x                                    x   
4                                    -                                    x   
5                                  998                                  998   
6                               by: f1                               by: f1   
7                                0.120                                0.172   

                                   (6)     rowtype  
0  -1.231*** <br> (0.192) <br> [0.000]        coef  
1     -0.074 <br> (0.104) <br> [0.482]        coef  
2     -0.041 <br> (0.081) <br> [0.618]        coef  
3                                    x          fe  
4                                    x          fe  
5                                  998  modelstats  
6                               by: f1  modelstats  
7                                0.172  modelstats  , _body=<great_tables._gt_data.Body object at 0x000001A1401EA050>, _boxhead=Boxhead([ColInfo(var='', type=<ColInfoTypeEnum.stub: 2>, column_label='', column_align='left', column_width=None), ColInfo(var='(1)', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='(2)', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='(3)', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='(4)', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='(5)', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='(6)', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None), ColInfo(var='rowtype', type=<ColInfoTypeEnum.row_group: 3>, column_label='rowtype', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001A1401EB430>, _spanners=Spanners([SpannerInfo(spanner_id='Y', spanner_level=1, spanner_label='Y', spanner_units=None, spanner_pattern=None, vars=['(1)', '(2)', '(3)'], built=None), SpannerInfo(spanner_id='Y2', spanner_level=1, spanner_label='Y2', spanner_units=None, spanner_pattern=None, vars=['(4)', '(5)', '(6)'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient c

### Significance levels and rounding
Additionally, we can also overwrite the defaults for the reported significance levels and control the rounding of results via the `signif_code` and `digits` function arguments: 

In [8]:
pf.etable([fit1, fit2, fit3, fit4, fit5, fit6], signif_code=[0.01, 0.05, 0.1], digits=5)

GT(_tbl_data=                                        (1)                         (2)  \
0            X1  -0.94953*** <br> (0.06652)  -0.92405*** <br> (0.06093)   
1            X2  -0.17423*** <br> (0.01840)  -0.17411*** <br> (0.01461)   
2         X1:X2                                                           
3            f1                           x                           x   
4            f2                           -                           x   
5  Observations                         997                         997   
6     S.E. type                      by: f1                      by: f1   
7            R2                     0.48899                     0.65904   

                          (3)                         (4)  \
0  -0.92417*** <br> (0.06094)  -1.26655*** <br> (0.17359)   
1  -0.18550*** <br> (0.02516)  -0.13056*** <br> (0.04239)   
2      0.01057 <br> (0.01818)                               
3                           x                           x   
4                           x                           -   
5                         997                         998   
6                      by: f1                      by: f1   
7                     0.65916                     0.12017   

                          (5)                         (6)     rowtype  
0  -1.23153*** <br> (0.19228)  -1.23100*** <br> (0.19167)        coef  
1  -0.11767*** <br> (0.04152)     -0.07369 <br> (0.10356)        coef  
2                                 -0.04082 <br> (0.08093)        coef  
3                           x                           x          fe  
4                           x                           x          fe  
5                         998                         998  modelstats  
6                      by: f1                      by: f1  modelstats  
7                     0.17151                     0.17180  modelstats  , _body=<great_tables._gt_data.Body object at 0x000001A14022CC70>, _boxhead=Boxhead([ColInfo(var='', type=<ColInfoTypeEnum.stub: 2>, column_label='', column_align='left', column_width=None), ColInfo(var='(1)', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='(2)', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='(3)', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='(4)', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='(5)', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='(6)', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None), ColInfo(var='rowtype', type=<ColInfoTypeEnum.row_group: 3>, column_label='rowtype', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001A14022E080>, _spanners=Spanners([SpannerInfo(spanner_id='Y', spanner_level=1, spanner_label='Y', spanner_units=None, spanner_pattern=None, vars=['(1)', '(2)', '(3)'], built=None), SpannerInfo(spanner_id='Y2', spanner_level=1, spanner_label='Y2', spanner_units=None, spanner_pattern=None, vars=['(4)', '(5)', '(6)'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.1, ** p < 0.05, *** p < 0.01. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001A14022D9C0>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInf

### Other output formats
By default, `pf.etable()` returns a GT object (see [the Great Tables package](https://posit-dev.github.io/great-tables/articles/intro.html)), but you can also opt to pandas styler, markdown, or latex output via the `type` argument.

In [9]:
# Pandas styler output:
pf.etable(
    [fit1, fit2, fit3, fit4, fit5, fit6],
    signif_code=[0.01, 0.05, 0.1],
    digits=5,
    type="df",
)

In [10]:
# Markdown output:
pf.etable(
    [fit1, fit2, fit3, fit4, fit5, fit6],
    signif_code=[0.01, 0.05, 0.1],
    digits=5,
    type="md",
)

                      est1          est2          est3          est4          est5          est6
------------  ------------  ------------  ------------  ------------  ------------  ------------
depvar                   Y             Y             Y            Y2            Y2            Y2
------------------------------------------------------------------------------------------------
X1            -0.94953***   -0.92405***   -0.92417***   -1.26655***   -1.23153***   -1.23100***
                 (0.06652)     (0.06093)     (0.06094)     (0.17359)     (0.19228)     (0.19167)
X2            -0.17423***   -0.17411***   -0.18550***   -0.13056***   -0.11767***      -0.07369
                 (0.01840)     (0.01461)     (0.02516)     (0.04239)     (0.04152)     (0.10356)
X1:X2                                         0.01057                                  -0.04082
                                             (0.01818)                                 (0.08093)
---------------------------------

To obtain latex output use `format = "tex"`. If you want to save the table as a tex file, you can use the `filename=` argument to specify the respective path where it should be saved. If you want the latex code to be displayed in the notebook, you can use the `print_tex=True` argument.
Etable will use latex packages `booktabs`, `threeparttable` and `makecell` for the table layout, so don't forget to include these packages in your latex document. 
 

In [11]:
# LaTex output (include latex packages booktabs, threeparttable, and makecell in your document):
tab = pf.etable(
    [fit1, fit2, fit3, fit4, fit5, fit6],
    signif_code=[0.01, 0.05, 0.1],
    digits=2,
    type="tex",
    print_tex=True,
)

\renewcommand\cellalign{t}
\begin{threeparttable}
\begin{tabular}{lcccccc}
\toprule
 & \multicolumn{3}{c}{Y} & \multicolumn{3}{c}{Y2} \\
\cmidrule(lr){2-4} \cmidrule(lr){5-7} 
 & (1) & (2) & (3) & (4) & (5) & (6) \\
\midrule
X1 & \makecell{-0.95*** \\ (0.07)} & \makecell{-0.92*** \\ (0.06)} & \makecell{-0.92*** \\ (0.06)} & \makecell{-1.27*** \\ (0.17)} & \makecell{-1.23*** \\ (0.19)} & \makecell{-1.23*** \\ (0.19)} \\
X2 & \makecell{-0.17*** \\ (0.02)} & \makecell{-0.17*** \\ (0.01)} & \makecell{-0.19*** \\ (0.03)} & \makecell{-0.13*** \\ (0.04)} & \makecell{-0.12*** \\ (0.04)} & \makecell{-0.07 \\ (0.10)} \\
X1:X2 &  &  & \makecell{0.01 \\ (0.02)} &  &  & \makecell{-0.04 \\ (0.08)} \\
\midrule
f1 & x & x & x & x & x & x \\
f2 & - & x & x & - & x & x \\
\midrule
Observations & 997 & 997 & 997 & 998 & 998 & 998 \\
S.E. type & by: f1 & by: f1 & by: f1 & by: f1 & by: f1 & by: f1 \\
$R^2$ & 0.49 & 0.66 & 0.66 & 0.12 & 0.17 & 0.17 \\
\bottomrule
\end{tabular}
\footnotesize Significance lev

The following code generates a pdf including the regression table which you can display clicking on the link below the cell:

In [12]:
## Use pylatex to create a tex file with the table


def make_pdf(tab, file):
    "Create a PDF document with tex table."
    doc = pl.Document()
    doc.packages.append(pl.Package("booktabs"))
    doc.packages.append(pl.Package("threeparttable"))
    doc.packages.append(pl.Package("makecell"))

    with (
        doc.create(pl.Section("A PyFixest LateX Table")),
        doc.create(pl.Table(position="htbp")) as table,
    ):
        table.append(pl.NoEscape(tab))

    doc.generate_pdf(file, clean_tex=False)


# Compile latex to pdf & display a button with the hyperlink to the pdf
make_pdf(tab, "latexdocs/SampleTableDoc")
display(FileLink("latexdocs/SampleTableDoc.pdf"))

c:\Users\alexa\Documents\pyfixest\docs\latexdocs\SampleTableDoc.pdf

### Custom Styling with Great Tables
You can use the rich set of methods offered by [Great Tables](https://posit-dev.github.io/great-tables/articles/intro.html) to further customize the table display.

In [13]:
(
    pf.etable([fit1, fit2, fit3, fit4, fit5, fit6])
    .tab_options(
        column_labels_background_color="cornsilk",
        stub_background_color="whitesmoke",
    )
    .tab_style(
        style=style.fill(color="mistyrose"),
        locations=loc.body(columns="(3)", rows=["X2"]),
    )
)

GT(_tbl_data=                                    (1)                     (2)  \
0            X1  -0.950*** <br> (0.067)  -0.924*** <br> (0.061)   
1            X2  -0.174*** <br> (0.018)  -0.174*** <br> (0.015)   
2         X1:X2                                                   
3            f1                       x                       x   
4            f2                       -                       x   
5  Observations                     997                     997   
6     S.E. type                  by: f1                  by: f1   
7            R2                   0.489                   0.659   

                      (3)                     (4)                     (5)  \
0  -0.924*** <br> (0.061)  -1.267*** <br> (0.174)  -1.232*** <br> (0.192)   
1  -0.185*** <br> (0.025)   -0.131** <br> (0.042)   -0.118** <br> (0.042)   
2      0.011 <br> (0.018)                                                   
3                       x                       x                       x   
4                       x                       -                       x   
5                     997                     998                     998   
6                  by: f1                  by: f1                  by: f1   
7                   0.659                   0.120                   0.172   

                      (6)     rowtype  
0  -1.231*** <br> (0.192)        coef  
1     -0.074 <br> (0.104)        coef  
2     -0.041 <br> (0.081)        coef  
3                       x          fe  
4                       x          fe  
5                     998  modelstats  
6                  by: f1  modelstats  
7                   0.172  modelstats  , _body=<great_tables._gt_data.Body object at 0x000001A140B98100>, _boxhead=Boxhead([ColInfo(var='', type=<ColInfoTypeEnum.stub: 2>, column_label='', column_align='left', column_width=None), ColInfo(var='(1)', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='(2)', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='(3)', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='(4)', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='(5)', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='(6)', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None), ColInfo(var='rowtype', type=<ColInfoTypeEnum.row_group: 3>, column_label='rowtype', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001A140B9AB90>, _spanners=Spanners([SpannerInfo(spanner_id='Y', spanner_level=1, spanner_label='Y', spanner_units=None, spanner_pattern=None, vars=['(1)', '(2)', '(3)'], built=None), SpannerInfo(spanner_id='Y2', spanner_level=1, spanner_label='Y2', spanner_units=None, spanner_pattern=None, vars=['(4)', '(5)', '(6)'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[StyleInfo(locname='data', locnum=5, grpname=None, colname='(3)', rownum=1, colnum=None, styles=[CellStyleFill(color='mistyrose')])], _locale=<great_tables._gt_data.Locale object at 0x000001A140B9B190>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsI

### Rename variables
You can also rename variables if you want to have a more readable output. Just pass a dictionary to the `labels` argument. Note that interaction terms will also be relabeled using the specified labels for the interacted variables (if you want to manually relabel an interaction term differently, add it to the dictionary).

In [14]:
labels = {
    "Y": "Wage",
    "Y2": "Wealth",
    "X1": "Age",
    "X2": "Years of Schooling",
    "f1": "Industry",
    "f2": "Year",
}

pf.etable([fit1, fit2, fit3, fit4, fit5, fit6], labels=labels)

GT(_tbl_data=                                                (1)                     (2)  \
0                       Age  -0.950*** <br> (0.067)  -0.924*** <br> (0.061)   
1        Years of Schooling  -0.174*** <br> (0.018)  -0.174*** <br> (0.015)   
2  Age x Years of Schooling                                                   
3                  Industry                       x                       x   
4                      Year                       -                       x   
5              Observations                     997                     997   
6                 S.E. type                  by: f1                  by: f1   
7                        R2                   0.489                   0.659   

                      (3)                     (4)                     (5)  \
0  -0.924*** <br> (0.061)  -1.267*** <br> (0.174)  -1.232*** <br> (0.192)   
1  -0.185*** <br> (0.025)   -0.131** <br> (0.042)   -0.118** <br> (0.042)   
2      0.011 <br> (0.018)                                                   
3                       x                       x                       x   
4                       x                       -                       x   
5                     997                     998                     998   
6                  by: f1                  by: f1                  by: f1   
7                   0.659                   0.120                   0.172   

                      (6)     rowtype  
0  -1.231*** <br> (0.192)        coef  
1     -0.074 <br> (0.104)        coef  
2     -0.041 <br> (0.081)        coef  
3                       x          fe  
4                       x          fe  
5                     998  modelstats  
6                  by: f1  modelstats  
7                   0.172  modelstats  , _body=<great_tables._gt_data.Body object at 0x000001A1400CC910>, _boxhead=Boxhead([ColInfo(var='', type=<ColInfoTypeEnum.stub: 2>, column_label='', column_align='left', column_width=None), ColInfo(var='(1)', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='(2)', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='(3)', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='(4)', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='(5)', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='(6)', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None), ColInfo(var='rowtype', type=<ColInfoTypeEnum.row_group: 3>, column_label='rowtype', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001A1400CE560>, _spanners=Spanners([SpannerInfo(spanner_id='Wage', spanner_level=1, spanner_label='Wage', spanner_units=None, spanner_pattern=None, vars=['(1)', '(2)', '(3)'], built=None), SpannerInfo(spanner_id='Wealth', spanner_level=1, spanner_label='Wealth', spanner_units=None, spanner_pattern=None, vars=['(4)', '(5)', '(6)'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001A1400CCDF0>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss

If you want to label the rows indicating the inclusion of fixed effects not with the variable label but with a custom label, you can pass on a separate dictionary to the `felabels` argument.

In [15]:
pf.etable(
    [fit1, fit2, fit3, fit4, fit5, fit6],
    labels=labels,
    felabels={"f1": "Industry Fixed Effects", "f2": "Year Fixed Effects"},
)

GT(_tbl_data=                                                (1)                     (2)  \
0                       Age  -0.950*** <br> (0.067)  -0.924*** <br> (0.061)   
1        Years of Schooling  -0.174*** <br> (0.018)  -0.174*** <br> (0.015)   
2  Age x Years of Schooling                                                   
3    Industry Fixed Effects                       x                       x   
4        Year Fixed Effects                       -                       x   
5              Observations                     997                     997   
6                 S.E. type                  by: f1                  by: f1   
7                        R2                   0.489                   0.659   

                      (3)                     (4)                     (5)  \
0  -0.924*** <br> (0.061)  -1.267*** <br> (0.174)  -1.232*** <br> (0.192)   
1  -0.185*** <br> (0.025)   -0.131** <br> (0.042)   -0.118** <br> (0.042)   
2      0.011 <br> (0.018)                                                   
3                       x                       x                       x   
4                       x                       -                       x   
5                     997                     998                     998   
6                  by: f1                  by: f1                  by: f1   
7                   0.659                   0.120                   0.172   

                      (6)     rowtype  
0  -1.231*** <br> (0.192)        coef  
1     -0.074 <br> (0.104)        coef  
2     -0.041 <br> (0.081)        coef  
3                       x          fe  
4                       x          fe  
5                     998  modelstats  
6                  by: f1  modelstats  
7                   0.172  modelstats  , _body=<great_tables._gt_data.Body object at 0x000001A1400CF250>, _boxhead=Boxhead([ColInfo(var='', type=<ColInfoTypeEnum.stub: 2>, column_label='', column_align='left', column_width=None), ColInfo(var='(1)', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='(2)', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='(3)', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='(4)', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='(5)', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='(6)', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None), ColInfo(var='rowtype', type=<ColInfoTypeEnum.row_group: 3>, column_label='rowtype', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001A14002E020>, _spanners=Spanners([SpannerInfo(spanner_id='Wage', spanner_level=1, spanner_label='Wage', spanner_units=None, spanner_pattern=None, vars=['(1)', '(2)', '(3)'], built=None), SpannerInfo(spanner_id='Wealth', spanner_level=1, spanner_label='Wealth', spanner_units=None, spanner_pattern=None, vars=['(4)', '(5)', '(6)'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001A1400CC7C0>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss

### Custom model headlines
You can also add custom headers for each model by passing a list of strings to the `model_headers` argument.

In [16]:
pf.etable(
    [fit1, fit2, fit3, fit4, fit5, fit6],
    labels=labels,
    model_heads=["US", "China", "EU", "US", "China", "EU"],
)

GT(_tbl_data=                                                (1)                     (2)  \
0                       Age  -0.950*** <br> (0.067)  -0.924*** <br> (0.061)   
1        Years of Schooling  -0.174*** <br> (0.018)  -0.174*** <br> (0.015)   
2  Age x Years of Schooling                                                   
3                  Industry                       x                       x   
4                      Year                       -                       x   
5              Observations                     997                     997   
6                 S.E. type                  by: f1                  by: f1   
7                        R2                   0.489                   0.659   

                      (3)                     (4)                     (5)  \
0  -0.924*** <br> (0.061)  -1.267*** <br> (0.174)  -1.232*** <br> (0.192)   
1  -0.185*** <br> (0.025)   -0.131** <br> (0.042)   -0.118** <br> (0.042)   
2      0.011 <br> (0.018)                                                   
3                       x                       x                       x   
4                       x                       -                       x   
5                     997                     998                     998   
6                  by: f1                  by: f1                  by: f1   
7                   0.659                   0.120                   0.172   

                      (6)     rowtype  
0  -1.231*** <br> (0.192)        coef  
1     -0.074 <br> (0.104)        coef  
2     -0.041 <br> (0.081)        coef  
3                       x          fe  
4                       x          fe  
5                     998  modelstats  
6                  by: f1  modelstats  
7                   0.172  modelstats  , _body=<great_tables._gt_data.Body object at 0x000001A14002CA60>, _boxhead=Boxhead([ColInfo(var='', type=<ColInfoTypeEnum.stub: 2>, column_label='', column_align='left', column_width=None), ColInfo(var='(1)', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='(2)', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='(3)', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='(4)', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='(5)', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='(6)', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None), ColInfo(var='rowtype', type=<ColInfoTypeEnum.row_group: 3>, column_label='rowtype', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001A14002CAF0>, _spanners=Spanners([SpannerInfo(spanner_id='Wage', spanner_level=2, spanner_label='Wage', spanner_units=None, spanner_pattern=None, vars=['(1)', '(2)', '(3)'], built=None), SpannerInfo(spanner_id='Wealth', spanner_level=2, spanner_label='Wealth', spanner_units=None, spanner_pattern=None, vars=['(4)', '(5)', '(6)'], built=None), SpannerInfo(spanner_id='US', spanner_level=1, spanner_label='US', spanner_units=None, spanner_pattern=None, vars=['(1)', '(4)'], built=None), SpannerInfo(spanner_id='China', spanner_level=1, spanner_label='China', spanner_units=None, spanner_pattern=None, vars=['(2)', '(5)'], built=None), SpannerInfo(spanner_id='EU', spanner_level=1, spanner_label='EU', spanner_units=None, spanner_pattern=None, vars=['(3)', '(6)'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001A14002DBD0>, _formats=[], _substitutions=[], _options=Options(table_id=Opt

Or change the ordering of headlines having headlines first and then dependent variables using the `head_order` argument. "hd" stands for headlines then dependent variables, "dh" for dependent variables then headlines. Assigning "d" or "h" can be used to only show dependent variables or only headlines. When head_order="" only model numbers are shown.

In [17]:
pf.etable(
    [fit1, fit4, fit2, fit5, fit3, fit6],
    labels=labels,
    model_heads=["US", "US", "China", "China", "EU", "EU"],
    head_order="hd",
)

GT(_tbl_data=                                                (1)                     (2)  \
0                       Age  -0.950*** <br> (0.067)  -1.267*** <br> (0.174)   
1        Years of Schooling  -0.174*** <br> (0.018)   -0.131** <br> (0.042)   
2  Age x Years of Schooling                                                   
3                  Industry                       x                       x   
4                      Year                       -                       -   
5              Observations                     997                     998   
6                 S.E. type                  by: f1                  by: f1   
7                        R2                   0.489                   0.120   

                      (3)                     (4)                     (5)  \
0  -0.924*** <br> (0.061)  -1.232*** <br> (0.192)  -0.924*** <br> (0.061)   
1  -0.174*** <br> (0.015)   -0.118** <br> (0.042)  -0.185*** <br> (0.025)   
2                                                      0.011 <br> (0.018)   
3                       x                       x                       x   
4                       x                       x                       x   
5                     997                     998                     997   
6                  by: f1                  by: f1                  by: f1   
7                   0.659                   0.172                   0.659   

                      (6)     rowtype  
0  -1.231*** <br> (0.192)        coef  
1     -0.074 <br> (0.104)        coef  
2     -0.041 <br> (0.081)        coef  
3                       x          fe  
4                       x          fe  
5                     998  modelstats  
6                  by: f1  modelstats  
7                   0.172  modelstats  , _body=<great_tables._gt_data.Body object at 0x000001A1400754B0>, _boxhead=Boxhead([ColInfo(var='', type=<ColInfoTypeEnum.stub: 2>, column_label='', column_align='left', column_width=None), ColInfo(var='(1)', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='(2)', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='(3)', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='(4)', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='(5)', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='(6)', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None), ColInfo(var='rowtype', type=<ColInfoTypeEnum.row_group: 3>, column_label='rowtype', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001A140076A40>, _spanners=Spanners([SpannerInfo(spanner_id='US', spanner_level=2, spanner_label='US', spanner_units=None, spanner_pattern=None, vars=['(1)', '(2)'], built=None), SpannerInfo(spanner_id='China', spanner_level=2, spanner_label='China', spanner_units=None, spanner_pattern=None, vars=['(3)', '(4)'], built=None), SpannerInfo(spanner_id='EU', spanner_level=2, spanner_label='EU', spanner_units=None, spanner_pattern=None, vars=['(5)', '(6)'], built=None), SpannerInfo(spanner_id='Wage', spanner_level=1, spanner_label='Wage', spanner_units=None, spanner_pattern=None, vars=['(1)', '(3)', '(5)'], built=None), SpannerInfo(spanner_id='Wealth', spanner_level=1, spanner_label='Wealth', spanner_units=None, spanner_pattern=None, vars=['(2)', '(4)', '(6)'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001A140076C80>, _formats=[], _substitutions=[], _options=Options(table_id=Opt

Remove the dependent variables from the headers:

In [18]:
pf.etable(
    [fit1, fit4, fit2, fit5, fit3, fit6],
    labels=labels,
    model_heads=["US", "US", "China", "China", "EU", "EU"],
    head_order="",
)

GT(_tbl_data=                                                (1)                     (2)  \
0                       Age  -0.950*** <br> (0.067)  -1.267*** <br> (0.174)   
1        Years of Schooling  -0.174*** <br> (0.018)   -0.131** <br> (0.042)   
2  Age x Years of Schooling                                                   
3                  Industry                       x                       x   
4                      Year                       -                       -   
5              Observations                     997                     998   
6                 S.E. type                  by: f1                  by: f1   
7                        R2                   0.489                   0.120   

                      (3)                     (4)                     (5)  \
0  -0.924*** <br> (0.061)  -1.232*** <br> (0.192)  -0.924*** <br> (0.061)   
1  -0.174*** <br> (0.015)   -0.118** <br> (0.042)  -0.185*** <br> (0.025)   
2                                                      0.011 <br> (0.018)   
3                       x                       x                       x   
4                       x                       x                       x   
5                     997                     998                     997   
6                  by: f1                  by: f1                  by: f1   
7                   0.659                   0.172                   0.659   

                      (6)     rowtype  
0  -1.231*** <br> (0.192)        coef  
1     -0.074 <br> (0.104)        coef  
2     -0.041 <br> (0.081)        coef  
3                       x          fe  
4                       x          fe  
5                     998  modelstats  
6                  by: f1  modelstats  
7                   0.172  modelstats  , _body=<great_tables._gt_data.Body object at 0x000001A140075090>, _boxhead=Boxhead([ColInfo(var='', type=<ColInfoTypeEnum.stub: 2>, column_label='', column_align='left', column_width=None), ColInfo(var='(1)', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='(2)', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='(3)', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='(4)', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='(5)', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='(6)', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None), ColInfo(var='rowtype', type=<ColInfoTypeEnum.row_group: 3>, column_label='rowtype', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001A14009C2E0>, _spanners=Spanners([]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001A14009CA00>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu',

### Further custom model information
You can add further custom model statistics/information to the bottom of the table by using the `custom_stats` argument to which you pass a dictionary with the name of the row and lists of values. The length of the lists must be equal to the number of models.

In [19]:
pf.etable(
    [fit1, fit2, fit3, fit4, fit5, fit6],
    labels=labels,
    custom_model_stats={
        "Number of Clusters": [42, 42, 42, 37, 37, 37],
        "Additional Info": ["A", "A", "B", "B", "C", "C"],
    },
)

GT(_tbl_data=                                                (1)                     (2)  \
0                       Age  -0.950*** <br> (0.067)  -0.924*** <br> (0.061)   
1        Years of Schooling  -0.174*** <br> (0.018)  -0.174*** <br> (0.015)   
2  Age x Years of Schooling                                                   
3                  Industry                       x                       x   
4                      Year                       -                       x   
5        Number of Clusters                      42                      42   
6           Additional Info                       A                       A   
7              Observations                     997                     997   
8                 S.E. type                  by: f1                  by: f1   
9                        R2                   0.489                   0.659   

                      (3)                     (4)                     (5)  \
0  -0.924*** <br> (0.061)  -1.267*** <br> (0.174)  -1.232*** <br> (0.192)   
1  -0.185*** <br> (0.025)   -0.131** <br> (0.042)   -0.118** <br> (0.042)   
2      0.011 <br> (0.018)                                                   
3                       x                       x                       x   
4                       x                       -                       x   
5                      42                      37                      37   
6                       B                       B                       C   
7                     997                     998                     998   
8                  by: f1                  by: f1                  by: f1   
9                   0.659                   0.120                   0.172   

                      (6)     rowtype  
0  -1.231*** <br> (0.192)        coef  
1     -0.074 <br> (0.104)        coef  
2     -0.041 <br> (0.081)        coef  
3                       x          fe  
4                       x          fe  
5                      37  modelstats  
6                       C  modelstats  
7                     998  modelstats  
8                  by: f1  modelstats  
9                   0.172  modelstats  , _body=<great_tables._gt_data.Body object at 0x000001A14009EAA0>, _boxhead=Boxhead([ColInfo(var='', type=<ColInfoTypeEnum.stub: 2>, column_label='', column_align='left', column_width=None), ColInfo(var='(1)', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='(2)', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='(3)', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='(4)', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='(5)', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='(6)', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None), ColInfo(var='rowtype', type=<ColInfoTypeEnum.row_group: 3>, column_label='rowtype', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001A140BC86D0>, _spanners=Spanners([SpannerInfo(spanner_id='Wage', spanner_level=1, spanner_label='Wage', spanner_units=None, spanner_pattern=None, vars=['(1)', '(2)', '(3)'], built=None), SpannerInfo(spanner_id='Wealth', spanner_level=1, spanner_label='Wealth', spanner_units=None, spanner_pattern=None, vars=['(4)', '(5)', '(6)'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001A14009E860>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', t

### Custom table notes
You can replace the default table notes with your own notes using the `notes` argument. 


In [20]:
mynotes = "Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit amet."
pf.etable(
    [fit1, fit4, fit2, fit5, fit3, fit6],
    labels=labels,
    model_heads=["US", "US", "China", "China", "EU", "EU"],
    head_order="hd",
    notes=mynotes,
)

GT(_tbl_data=                                                (1)                     (2)  \
0                       Age  -0.950*** <br> (0.067)  -1.267*** <br> (0.174)   
1        Years of Schooling  -0.174*** <br> (0.018)   -0.131** <br> (0.042)   
2  Age x Years of Schooling                                                   
3                  Industry                       x                       x   
4                      Year                       -                       -   
5              Observations                     997                     998   
6                 S.E. type                  by: f1                  by: f1   
7                        R2                   0.489                   0.120   

                      (3)                     (4)                     (5)  \
0  -0.924*** <br> (0.061)  -1.232*** <br> (0.192)  -0.924*** <br> (0.061)   
1  -0.174*** <br> (0.015)   -0.118** <br> (0.042)  -0.185*** <br> (0.025)   
2                                                      0.011 <br> (0.018)   
3                       x                       x                       x   
4                       x                       x                       x   
5                     997                     998                     997   
6                  by: f1                  by: f1                  by: f1   
7                   0.659                   0.172                   0.659   

                      (6)     rowtype  
0  -1.231*** <br> (0.192)        coef  
1     -0.074 <br> (0.104)        coef  
2     -0.041 <br> (0.081)        coef  
3                       x          fe  
4                       x          fe  
5                     998  modelstats  
6                  by: f1  modelstats  
7                   0.172  modelstats  , _body=<great_tables._gt_data.Body object at 0x000001A140BC9C90>, _boxhead=Boxhead([ColInfo(var='', type=<ColInfoTypeEnum.stub: 2>, column_label='', column_align='left', column_width=None), ColInfo(var='(1)', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='(2)', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='(3)', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='(4)', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='(5)', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='(6)', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None), ColInfo(var='rowtype', type=<ColInfoTypeEnum.row_group: 3>, column_label='rowtype', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001A140BCB130>, _spanners=Spanners([SpannerInfo(spanner_id='US', spanner_level=2, spanner_label='US', spanner_units=None, spanner_pattern=None, vars=['(1)', '(2)'], built=None), SpannerInfo(spanner_id='China', spanner_level=2, spanner_label='China', spanner_units=None, spanner_pattern=None, vars=['(3)', '(4)'], built=None), SpannerInfo(spanner_id='EU', spanner_level=2, spanner_label='EU', spanner_units=None, spanner_pattern=None, vars=['(5)', '(6)'], built=None), SpannerInfo(spanner_id='Wage', spanner_level=1, spanner_label='Wage', spanner_units=None, spanner_pattern=None, vars=['(1)', '(3)', '(5)'], built=None), SpannerInfo(spanner_id='Wealth', spanner_level=1, spanner_label='Wealth', spanner_units=None, spanner_pattern=None, vars=['(2)', '(4)', '(6)'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsu

### Publication-ready LaTex tables 
With few lines of code you thus obtain a publication-ready latex table:

In [21]:
tab = pf.etable(
    [fit1, fit4, fit2, fit5, fit3, fit6],
    labels=labels,
    model_heads=["US", "US", "China", "China", "EU", "EU"],
    head_order="hd",
    type="tex",
    notes=mynotes,
    show_fe=True,
    show_se_type=False,
    custom_model_stats={
        "Number of Clusters": [42, 42, 42, 37, 37, 37],
    },
)

# Compile latex to pdf & display a button with the hyperlink to the pdf
make_pdf(tab, "latexdocs/SampleTableDoc2")
display(FileLink("latexdocs/SampleTableDoc2.pdf"))

c:\Users\alexa\Documents\pyfixest\docs\latexdocs\SampleTableDoc2.pdf

### Rendering Tables in Quarto
When you use quarto you can include latex tables generated by `pf.etable` when rendering the qmd file as pdf. Just specify `output: asis` in the code block options of the respective chunk and print the LaTex string returned by etable. Don't forget to include the `\usepackage` commands for necessary latex packages in the YAML block.

Here you find a sample [qmd file](https://github.com/py-econometrics/pyfixest/blob/master/docs/quarto_example/QuartoExample.qmd) and the rendered [PDF](quarto_example/QuartoExample.pdf).

## Regression Tables via `Stargazer`

We have opened a PR for `pyfixest` support for the excellent [Stargazer](https://github.com/StatsReporting/stargazer/pull/105) project. Until it is merged, you can download the dev version from `py-econometrics` by typing

```bash
pip install git+https://github.com/py-econometrics/stargazer.git
```

`Stargazer` is particularly useful if you need highly customizable regression tables (beyond the scope of `pf.etable()`), or if you want to compare models from `statsmodels` or `linearmodels` with `pyfixest`. 

After installing `stargazer`, we can produce a summary table via the `Stargazer` class: 

In [22]:
stargazer_table = Stargazer([fit1, fit2, fit3, fit4, fit5, fit6])
stargazer_table

We can easily add custom statisics. For example, assume that we want to correct for multiple testing via the Romano-Wolf correction. We can do this as follows:

In [23]:
rwolf_res = pf.rwolf(
    [fit1, fit2, fit3, fit4, fit5, fit6], param="X1", seed=123, reps=9999
)
rwolf_pvalues = np.round(rwolf_res.xs("RW Pr(>|t|)"), 3).to_list()

In [24]:
stargazer_table.add_line(
    "Fixed Effects",
    [x._fixef for x in [fit1, fit2, fit3, fit4, fit5, fit6]],
    LineLocation.FOOTER_TOP,
)
stargazer_table.add_line(
    "X1: Romano-Wolf P-Value", rwolf_pvalues, LineLocation.FOOTER_TOP
)
stargazer_table